Installing the Sharpcosine library for Sharpened Cosine Similarity Convolution layer and Absolute Max Pooling layer

In [1]:
!pip install sharpcosine==0.1.1

Downloading the Kaggle dataset for the experiment

In [2]:
from google.colab import files
files.upload() # uploading the Kaggle API

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sleepingcat4","key":"88047ec16dbf3ab3c2aa1dbdf9c108d7"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d pmigdal/alien-vs-predator-images

 85% 12.0M/14.1M [00:00<00:00, 20.6MB/s]
100% 14.1M/14.1M [00:00<00:00, 15.8MB/s]


Importing the required libraries

In [5]:
# Importing the sharp modules
from sharpcosine import CosSimConv2D
from sharpcosine import MaxAbsPool2D

In [12]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras import Model, layers
import tensorflow as tf

from tensorflow.keras.layers import InputLayer, Dense, Flatten
from tensorflow.keras.models import Sequential

In [7]:
!unzip /content/alien-vs-predator-images.zip

Archive:  /content/alien-vs-predator-images.zip
  inflating: alien_vs_predator_thumbnails/data/train/alien/0.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/1.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/10.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/100.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/101.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/102.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/103.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/104.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/105.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/106.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/107.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/108.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/109.jpg  
  inflating: alien_vs_predator_thumbnails/data/train/alien/1

In [8]:
dataset_path = "/content/data"

In [22]:
train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_ds = train_datagen.flow_from_directory(
    dataset_path + '/train',
    batch_size=32,
    class_mode='binary',
    target_size=(224, 224))

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

validation_ds = validation_datagen.flow_from_directory(
    dataset_path + '/validation',
    shuffle=False,
    class_mode='binary',
    target_size=(224, 224))

Found 694 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [23]:
base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

In [24]:
model = tf.keras.Sequential(
    [
        base_model,
        CosSimConv2D(3),
        MaxAbsPool2D(2, True),
        CosSimConv2D(3),
        MaxAbsPool2D(2, True),
        CosSimConv2D(3),
        MaxAbsPool2D(2, True),
        Flatten(),
        Dense(1)
    ]
)

In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 cos_sim_conv2d_9 (CosSimCo  (None, 7, 7, 3)           55300     
 nv2D)                                                           
                                                                 
 max_abs_pool2d_9 (MaxAbsPo  (None, 4, 4, 3)           0         
 ol2D)                                                           
                                                                 
 cos_sim_conv2d_10 (CosSimC  (None, 4, 4, 3)           85        
 onv2D)                                                          
                                                                 
 max_abs_pool2d_10 (MaxAbsP  (None, 2, 2, 3)           0         
 ool2D)                                               

In [27]:
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, weight_decay=0.005),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

In [28]:
epochs = 50
print("Fitting the model")
iteration = model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

Fitting the model
Epoch 1/50
22/22 [==============================] - 22s 632ms/step - loss: 0.7252 - binary_accuracy: 0.4712 - val_loss: 0.6895 - val_binary_accuracy: 0.5250
Epoch 2/50
22/22 [==============================] - 10s 456ms/step - loss: 0.6570 - binary_accuracy: 0.5331 - val_loss: 0.6361 - val_binary_accuracy: 0.5450
Epoch 3/50
22/22 [==============================] - 10s 458ms/step - loss: 0.5805 - binary_accuracy: 0.6729 - val_loss: 0.5714 - val_binary_accuracy: 0.7250
Epoch 4/50
22/22 [==============================] - 10s 438ms/step - loss: 0.5153 - binary_accuracy: 0.7911 - val_loss: 0.5110 - val_binary_accuracy: 0.7600
Epoch 5/50
22/22 [==============================] - 9s 408ms/step - loss: 0.4203 - binary_accuracy: 0.8415 - val_loss: 0.4516 - val_binary_accuracy: 0.8050
Epoch 6/50
22/22 [==============================] - 11s 472ms/step - loss: 0.3757 - binary_accuracy: 0.8732 - val_loss: 0.4528 - val_binary_accuracy: 0.8050
Epoch 7/50
22/22 [=======================